#### MovieReview feature -  depends on ratings provieded by users and the papularity of  the movie ie number of users 

In [61]:
# DataSource :  https://grouplens.org/datasets/movielens/latest/
# Details: http://files.grouplens.org/datasets/movielens/ml-latest-README.html
# It contains 27753444 ratings and 1108997 tag applications across 58098 movies. 
# These data were created by 283228 users between January 09, 1995 and September 26, 2018. 
# This dataset was generated on September 26, 2018. 
# movies.csv , ratings.csv, tags.csv, links.csv ,genome-scores.csv ,genome-tags.csv

In [62]:
# During deployment of this model in to heroko seeing problem due to huge size of modle and trained vector
# model size give approxmatly 200MB this much huge data not accepted in github during commt time also
# we are going to reduce this dataset size and number of year of movies

# collect data from 2013 to till 2018 - 10 yrs movies 
# instead of 20 years data we taking 10yrs data

In [63]:
# import required libraries
import pandas as pd
import numpy as np

In [64]:
# Load Movies 
movies_df= pd.read_csv('ml-latest/movies.csv',usecols=['movieId','title','genres'],dtype={'movieId': 'int32', 'title': 'str','genres': 'str'})
movies_df


,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
58093,193876,The Great Glinka (1946),(no genres listed)
58094,193878,Les tribulations d'une caissière (2011),Comedy
58095,193880,Her Name Was Mumu (2016),Drama
58096,193882,Flora (2017),Adventure|Drama|Horror|Sci-Fi


In [65]:
def getTitle(title):
    try:
        _title=title.split("(")[0]
    except:
        _title="No_Title"
    return _title



In [66]:
# If year is unknown set it as 2008
def getYear(title):
    try:
        _year=title.split("(")[1].split(")")[0]
    except:
        _year =2013
    return _year

In [67]:
def parseGenres(genres):
    try:
        genre= genres.replace("|", " , ")
    except:
        genre="No genres"
    return genre


In [68]:
# Testing parseGenres
parseGenres("Adventure|Animation|Children|Comedy|Fantasy")

'Adventure , Animation , Children , Comedy , Fantasy'

In [69]:
# Testing parseGenres
parseGenres("(no genres listed)")

'(no genres listed)'

In [70]:
movies_df["title"]=movies_df["title"].str.strip()

In [71]:
movies_df["title"]=movies_df["title"].str.lower()

In [72]:
movies_df["genres"]=movies_df["genres"].str.strip()

In [73]:
movies_df["genres"]=movies_df["genres"].str.lower()

In [74]:
movies_df["year"]=movies_df["title"].apply(getYear)

In [75]:
movies_df["title"]=movies_df["title"].apply(getTitle)

In [76]:
movies_df["genres"]=movies_df["genres"].apply(parseGenres)

In [77]:
movies_df

,movieId,title,genres,year
0,1,toy story,"adventure , animation , children , comedy , fa...",1995
1,2,jumanji,"adventure , children , fantasy",1995
2,3,grumpier old men,"comedy , romance",1995
3,4,waiting to exhale,"comedy , drama , romance",1995
4,5,father of the bride part ii,comedy,1995
...,...,...,...,...
58093,193876,the great glinka,(no genres listed),1946
58094,193878,les tribulations d'une caissière,comedy,2011
58095,193880,her name was mumu,drama,2016
58096,193882,flora,"adventure , drama , horror , sci-fi",2017


In [78]:
#Load Movie Ratings
ratings_df= pd.read_csv('ml-latest/ratings.csv',usecols=['userId','movieId','rating','timestamp'],dtype={'userId': 'int32', 'movieId': 'int32', 'rating': 'float32', 'timestamp': 'str'})
ratings_df

,userId,movieId,rating,timestamp
0,1,307,3.5,1256677221
1,1,481,3.5,1256677456
2,1,1091,1.5,1256677471
3,1,1257,4.5,1256677460
4,1,1449,4.5,1256677264
...,...,...,...,...
27753439,283228,8542,4.5,1379882795
27753440,283228,8712,4.5,1379882751
27753441,283228,34405,4.5,1379882889
27753442,283228,44761,4.5,1354159524


In [79]:
#Load movies with resource links 
links_df= pd.read_csv('ml-latest/links.csv',usecols=['movieId','imdbId','tmdbId'],dtype={'movieId': 'int32', 'imdbId': 'int32','tmdbId': 'str'})
links_df

,movieId,imdbId,tmdbId
0,1,114709,862
1,2,113497,8844
2,3,113228,15602
3,4,114885,31357
4,5,113041,11862
...,...,...,...
58093,193876,38566,78251
58094,193878,1754787,87558
58095,193880,5847740,422666
58096,193882,4453756,454439


In [80]:
# Movielens - https://movielens.org/movies/{movieId}
# IMDB      - http://www.imdb.com/title/{imdbId}/ 
# TMDID     - https://www.themoviedb.org/movie/{tmdbId}

In [81]:
def movieLensLink(id):
    link = "https://movielens.org/movies/{0}".format(id)
    return link

In [82]:
def imdbLink(id):
    link = "http://www.imdb.com/title/{0}/".format(id)
    return link

In [83]:
def tmdLink(id):
    link = "https://www.themoviedb.org/movie/{0}".format(id)
    return link

In [84]:
links_df["movieLensLink"]=links_df["movieId"].apply(movieLensLink)

In [85]:
links_df["imdbLink"]=links_df["imdbId"].apply(imdbLink)

In [86]:
links_df["tmdLink"]=links_df["tmdbId"].apply(tmdLink)

In [87]:
links_df

,movieId,imdbId,tmdbId,movieLensLink,imdbLink,tmdLink
0,1,114709,862,https://movielens.org/movies/1,http://www.imdb.com/title/114709/,https://www.themoviedb.org/movie/862
1,2,113497,8844,https://movielens.org/movies/2,http://www.imdb.com/title/113497/,https://www.themoviedb.org/movie/8844
2,3,113228,15602,https://movielens.org/movies/3,http://www.imdb.com/title/113228/,https://www.themoviedb.org/movie/15602
3,4,114885,31357,https://movielens.org/movies/4,http://www.imdb.com/title/114885/,https://www.themoviedb.org/movie/31357
4,5,113041,11862,https://movielens.org/movies/5,http://www.imdb.com/title/113041/,https://www.themoviedb.org/movie/11862
...,...,...,...,...,...,...
58093,193876,38566,78251,https://movielens.org/movies/193876,http://www.imdb.com/title/38566/,https://www.themoviedb.org/movie/78251
58094,193878,1754787,87558,https://movielens.org/movies/193878,http://www.imdb.com/title/1754787/,https://www.themoviedb.org/movie/87558
58095,193880,5847740,422666,https://movielens.org/movies/193880,http://www.imdb.com/title/5847740/,https://www.themoviedb.org/movie/422666
58096,193882,4453756,454439,https://movielens.org/movies/193882,http://www.imdb.com/title/4453756/,https://www.themoviedb.org/movie/454439


In [88]:
# https://movielens.org/movies/1        - Not accessible due to security  
# https://www.imdb.com/title/114709/    - Not accessible due to security
# https://www.themoviedb.org/movie/862  - TMD is Working 

In [89]:
# Drop unwanted columns
# imdbId tmdbId movieLensLink imdbLink
links_df_final= links_df.drop(columns=["imdbId","tmdbId","movieLensLink","imdbLink"],axis=1)
links_df_final

,movieId,tmdLink
0,1,https://www.themoviedb.org/movie/862
1,2,https://www.themoviedb.org/movie/8844
2,3,https://www.themoviedb.org/movie/15602
3,4,https://www.themoviedb.org/movie/31357
4,5,https://www.themoviedb.org/movie/11862
...,...,...
58093,193876,https://www.themoviedb.org/movie/78251
58094,193878,https://www.themoviedb.org/movie/87558
58095,193880,https://www.themoviedb.org/movie/422666
58096,193882,https://www.themoviedb.org/movie/454439


In [90]:
# check is there duplicate rows 58098 - all are unique
links_df_final.duplicated(subset=None, keep='first')

0        False
1        False
2        False
3        False
4        False
         ...  
58093    False
58094    False
58095    False
58096    False
58097    False
Length: 58098, dtype: bool

In [91]:
# find movie ratings total counts
ratings_df_count = ratings_df.groupby(by=["movieId"])["userId"].count().reset_index()
ratings_df_count

,movieId,userId
0,1,68469
1,2,27143
2,3,15585
3,4,2989
4,5,15474
...,...,...
53884,193876,1
53885,193878,1
53886,193880,1
53887,193882,1


In [92]:
#remove timestamp column, we are not using
ratings_df.drop(columns=["timestamp"],axis=1,inplace=True)
ratings_df

,userId,movieId,rating
0,1,307,3.5
1,1,481,3.5
2,1,1091,1.5
3,1,1257,4.5
4,1,1449,4.5
...,...,...,...
27753439,283228,8542,4.5
27753440,283228,8712,4.5
27753441,283228,34405,4.5
27753442,283228,44761,4.5


In [93]:
# find movie ratings total counts with respect to each rating
#ratings_df_count = ratings_df.groupby(by=["movieId"])["rating"].count().reset_index()

ratings_df_1=ratings_df.groupby(by=["movieId","rating"]).count().reset_index()
ratings_df_1

,movieId,rating,userId
0,1,0.5,478
1,1,1.0,840
2,1,1.5,468
3,1,2.0,2160
4,1,2.5,1665
...,...,...,...
272281,193878,2.0,1
272282,193880,2.0,1
272283,193882,2.0,1
272284,193886,2.5,1


In [94]:
# find movie average rating with respect to totalcount
pd.options.display.float_format = '{:,.1f}'.format
ratings_df_avg=ratings_df.groupby(by=["movieId"])["rating"].mean().reset_index()
ratings_df_avg

,movieId,rating
0,1,3.9
1,2,3.2
2,3,3.2
3,4,2.9
4,5,3.1
...,...,...
53884,193876,3.0
53885,193878,2.0
53886,193880,2.0
53887,193882,2.0


In [95]:
ratings_df_avg.rename(columns={"rating":"avg_rating"},inplace=True)
ratings_df_avg

,movieId,avg_rating
0,1,3.9
1,2,3.2
2,3,3.2
3,4,2.9
4,5,3.1
...,...,...
53884,193876,3.0
53885,193878,2.0
53886,193880,2.0
53887,193882,2.0


In [96]:
# display each rating counts in a row with respect to movie id
ratings_df_pivot=ratings_df_1.pivot(index='movieId', columns='rating',values='userId').fillna(0).astype(int).reset_index()
#print columns
print(ratings_df_pivot.columns)
ratings_df_pivot

Index(['movieId', 0.5, 1.0, 1.5, 2.0, 2.5, 3.0, 3.5, 4.0, 4.5, 5.0], dtype='object', name='rating')


rating,movieId,0.5,1.0,1.5,2.0,2.5,3.0,3.5,4.0,4.5,5.0
0,1,478,840,468,2160,1665,11872,6095,22710,5684,16497
1,2,276,813,542,2551,1902,8711,3116,6303,790,2139
2,3,174,777,237,1677,629,6227,774,3440,159,1491
3,4,48,352,31,497,77,1136,68,557,12,211
4,5,248,802,282,1737,681,6655,761,2989,174,1145
...,...,...,...,...,...,...,...,...,...,...,...
53884,193876,0,0,0,0,0,1,0,0,0,0
53885,193878,0,0,0,1,0,0,0,0,0,0
53886,193880,0,0,0,1,0,0,0,0,0,0
53887,193882,0,0,0,1,0,0,0,0,0,0


In [97]:
# Merge ratings total rating count and each rating count
ratings_df_final=pd.merge(ratings_df_count,ratings_df_pivot, on="movieId", how ="inner") 

In [98]:
ratings_df_final.rename( columns={"userId":"total_rating"},inplace=True)

In [99]:
ratings_df_final

,movieId,total_rating,0.5,1.0,1.5,2.0,2.5,3.0,3.5,4.0,4.5,5.0
0,1,68469,478,840,468,2160,1665,11872,6095,22710,5684,16497
1,2,27143,276,813,542,2551,1902,8711,3116,6303,790,2139
2,3,15585,174,777,237,1677,629,6227,774,3440,159,1491
3,4,2989,48,352,31,497,77,1136,68,557,12,211
4,5,15474,248,802,282,1737,681,6655,761,2989,174,1145
...,...,...,...,...,...,...,...,...,...,...,...,...
53884,193876,1,0,0,0,0,0,1,0,0,0,0
53885,193878,1,0,0,0,1,0,0,0,0,0,0
53886,193880,1,0,0,0,1,0,0,0,0,0,0
53887,193882,1,0,0,0,1,0,0,0,0,0,0


In [100]:
# Merge add avag rating to ratings_df_final
ratings_df_final=pd.merge(ratings_df_final,ratings_df_avg, on="movieId", how ="inner") 
ratings_df_final

,movieId,total_rating,0.5,1.0,1.5,2.0,2.5,3.0,3.5,4.0,4.5,5.0,avg_rating
0,1,68469,478,840,468,2160,1665,11872,6095,22710,5684,16497,3.9
1,2,27143,276,813,542,2551,1902,8711,3116,6303,790,2139,3.2
2,3,15585,174,777,237,1677,629,6227,774,3440,159,1491,3.2
3,4,2989,48,352,31,497,77,1136,68,557,12,211,2.9
4,5,15474,248,802,282,1737,681,6655,761,2989,174,1145,3.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
53884,193876,1,0,0,0,0,0,1,0,0,0,0,3.0
53885,193878,1,0,0,0,1,0,0,0,0,0,0,2.0
53886,193880,1,0,0,0,1,0,0,0,0,0,0,2.0
53887,193882,1,0,0,0,1,0,0,0,0,0,0,2.0


In [101]:
#Null check of move dataframe
movies_df.isnull().sum()

movieId    0
title      0
genres     0
year       0
dtype: int64

#### Finalize Dataset - need to merge movies_df , links_df_final , ratings_df_final

In [102]:
movie_links_df=pd.merge(movies_df,links_df_final, on="movieId", how ="inner") 

In [103]:
movie_links_df

,movieId,title,genres,year,tmdLink
0,1,toy story,"adventure , animation , children , comedy , fa...",1995,https://www.themoviedb.org/movie/862
1,2,jumanji,"adventure , children , fantasy",1995,https://www.themoviedb.org/movie/8844
2,3,grumpier old men,"comedy , romance",1995,https://www.themoviedb.org/movie/15602
3,4,waiting to exhale,"comedy , drama , romance",1995,https://www.themoviedb.org/movie/31357
4,5,father of the bride part ii,comedy,1995,https://www.themoviedb.org/movie/11862
...,...,...,...,...,...
58093,193876,the great glinka,(no genres listed),1946,https://www.themoviedb.org/movie/78251
58094,193878,les tribulations d'une caissière,comedy,2011,https://www.themoviedb.org/movie/87558
58095,193880,her name was mumu,drama,2016,https://www.themoviedb.org/movie/422666
58096,193882,flora,"adventure , drama , horror , sci-fi",2017,https://www.themoviedb.org/movie/454439


In [104]:
movie_links_ratings_final=pd.merge(movie_links_df,ratings_df_final, on="movieId", how ="inner") 

In [105]:
movie_links_ratings_final

,movieId,title,genres,year,tmdLink,total_rating,0.5,1.0,1.5,2.0,2.5,3.0,3.5,4.0,4.5,5.0,avg_rating
0,1,toy story,"adventure , animation , children , comedy , fa...",1995,https://www.themoviedb.org/movie/862,68469,478,840,468,2160,1665,11872,6095,22710,5684,16497,3.9
1,2,jumanji,"adventure , children , fantasy",1995,https://www.themoviedb.org/movie/8844,27143,276,813,542,2551,1902,8711,3116,6303,790,2139,3.2
2,3,grumpier old men,"comedy , romance",1995,https://www.themoviedb.org/movie/15602,15585,174,777,237,1677,629,6227,774,3440,159,1491,3.2
3,4,waiting to exhale,"comedy , drama , romance",1995,https://www.themoviedb.org/movie/31357,2989,48,352,31,497,77,1136,68,557,12,211,2.9
4,5,father of the bride part ii,comedy,1995,https://www.themoviedb.org/movie/11862,15474,248,802,282,1737,681,6655,761,2989,174,1145,3.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
53884,193876,the great glinka,(no genres listed),1946,https://www.themoviedb.org/movie/78251,1,0,0,0,0,0,1,0,0,0,0,3.0
53885,193878,les tribulations d'une caissière,comedy,2011,https://www.themoviedb.org/movie/87558,1,0,0,0,1,0,0,0,0,0,0,2.0
53886,193880,her name was mumu,drama,2016,https://www.themoviedb.org/movie/422666,1,0,0,0,1,0,0,0,0,0,0,2.0
53887,193882,flora,"adventure , drama , horror , sci-fi",2017,https://www.themoviedb.org/movie/454439,1,0,0,0,1,0,0,0,0,0,0,2.0


In [106]:
# Null check
movie_links_ratings_final.isnull().sum()

movieId         0
title           0
genres          0
year            0
tmdLink         0
total_rating    0
0.5             0
1.0             0
1.5             0
2.0             0
2.5             0
3.0             0
3.5             0
4.0             0
4.5             0
5.0             0
avg_rating      0
dtype: int64

In [107]:
#movie_links_ratings_final.year = pd.to_numeric(movie_links_ratings_final.year, errors='coerce').fillna(0).astype(np.int64)
#movie_links_ratings_final[movie_links_ratings_final["year"]==1995]

In [108]:
# need to convert year string to numeric datatype to run queries
movie_links_ratings_final.year = pd.to_numeric(movie_links_ratings_final.year, errors='coerce').fillna(0).astype(np.int64)
movie_links_ratings_final_filtered=movie_links_ratings_final[movie_links_ratings_final["year"]>=2013]


In [109]:
movie_links_ratings_final.shape

(53889, 17)

In [110]:
movie_links_ratings_final_filtered.shape

(11436, 17)

####  Save this movie_links_ratings_final as source dataset for next phase

In [111]:
movie_links_ratings_final_filtered.to_csv('move_review_dataset.csv', index=False,encoding='utf-8',compression='gzip')

In [112]:
movie_links_ratings_final_filtered

,movieId,title,genres,year,tmdLink,total_rating,0.5,1.0,1.5,2.0,2.5,3.0,3.5,4.0,4.5,5.0,avg_rating
10018,32930,category 6: day of destruction,"action , drama",2013,https://www.themoviedb.org/movie/32128,3,0,0,1,0,0,1,0,0,0,1,3.2
10608,40697,babylon 5,sci-fi,2013,https://www.themoviedb.org/movie/nan,200,13,1,2,4,5,12,28,48,26,61,3.9
19319,95595,bela kiss: prologue,"horror , mystery , thriller",2013,https://www.themoviedb.org/movie/155288,8,0,2,0,1,1,2,0,1,0,1,2.7
20205,99007,warm bodies,"comedy , horror , romance",2013,https://www.themoviedb.org/movie/82654,2168,51,44,60,141,188,440,456,459,173,156,3.3
20283,99335,addicted,drama,2013,https://www.themoviedb.org/movie/235271,13,0,1,1,1,1,1,3,4,0,1,3.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
53879,193866,tales from the hood 2,horror,2018,https://www.themoviedb.org/movie/470614,1,0,1,0,0,0,0,0,0,0,0,1.0
53882,193872,room laundering,drama,2018,https://www.themoviedb.org/movie/532677,1,0,0,0,0,0,0,0,1,0,0,4.0
53886,193880,her name was mumu,drama,2016,https://www.themoviedb.org/movie/422666,1,0,0,0,1,0,0,0,0,0,0,2.0
53887,193882,flora,"adventure , drama , horror , sci-fi",2017,https://www.themoviedb.org/movie/454439,1,0,0,0,1,0,0,0,0,0,0,2.0
